<a id="top"></a>
# Land Change

<hr>

*NOT currently compatible with the DE Africa environment

> A notebook that demonstrates using Fractional Cover to detect vegetation change is available under the `Real_world_examples` folder: [Vegetation_change_detection](../../Real_world_examples/Vegetation_change_detection.ipynb)

# Notebook Summary

This notebook uses changes in NDVI, EVI or Fractional Cover to identify land change. 
The algorithm identifies a "baseline" and "analysis" time period and then compares 
the spectral parameters in each of those time periods. Significant reductions in 
vegetation are coincident with land change. In some cases these changes could be 
deforestation. Users of this algorithm should not accept the accuracy of the results 
but should conduct ground validation testing to assess accuracy. In most cases, these 
algorithms can be used to identify clusters of pixels that have experienced change and 
allow targeted investigation of those areas by local or regional governments.

<hr>

# Index

* [Import Dependencies and Connect to the Data Cube](#import)
* [Choose Platforms and Products](#plat_prod)
* [Get the Extents of the Cube](#extents)
* [Define the Extents of the Analysis](#define_extents)
* [Define Analysis Parameters](#define_analysis_params)
* [Load and Clean Data from the Data Cube](#load_data)
* [Calculate Spectral Indices and Fractional Cover](#calculate)
* [Show Threshold Image](#threshold)
* [Show Anomalies](#anomalies)
    * Show Spectral Anomaly Product
    * Show Threshold Spectral Anomaly Product
* [Create GeoTIFF Output Products](#export)

## <span id="import">Import Dependencies and Connect to the Data Cube [&#9652;](#top)</span>

In [ ]:
# Load Data Cube Configuration
import datacube
# Enable importing of utilities.
import sys
import os

# Supress Warnings.
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.cm import RdYlGn, Greens
import dask
from datacube.utils.dask import start_local_dask
from datacube.utils.rio import configure_s3_access
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from odc.ui import DcViewer
from datacube.helpers import write_geotiff
import datetime as dt

#import DE Africa script
sys.path.append('../../Scripts')
from deafrica_plotting import display_map
from deafrica_plotting import rgb


#import DCAL utility scripts
sys.path.append('../DCAL_utils')
from plotter_utils import figure_ratio, retrieve_or_create_fig_ax, intersection_threshold_plot, month_names_long
from dc_mosaic import create_max_ndvi_mosaic, create_min_ndvi_mosaic, create_median_mosaic, create_mosaic, create_hdmedians_multiple_band_mosaic, ls8_unpack_qa, ls7_unpack_qa 
from dc_time import _n64_to_datetime
from clean_mask import landsat_qa_clean_mask
from dc_fractional_coverage_classifier import frac_coverage_classify


## Set up a dask cluster
This will help keep our memory use down and conduct the analysis in parallel. If you'd like to view the dask dashboard, click on the hyperlink that prints below the cell.

In [ ]:
dask.config.set({"distributed.dashboard.link": 
        os.environ.get('JUPYTERHUB_SERVICE_PREFIX', '/')+"proxy/{port}/status"})

client = start_local_dask(n_workers=4, threads_per_worker=1, memory_limit='3G')
display(client)

creds = configure_s3_access(client=client, 
                            region_name='auto')

## <span id="plat_prod">Choose Platforms and Products [&#9652;](#top)</span>

**Visualize the available area**

In [ ]:
dc = datacube.Datacube(app="DCAL Land Change")

In [ ]:
DcViewer(dc=dc,
         products = ['ls8_usgs_sr_scene'],
         time='2017',
         center=(0.565, 38.007),
         zoom=4)

## <span id="define_extents">Define the Extents of the Analysis [&#9652;](#top)</span>

<p style="color:red";><b>CHANGE INPUTS BELOW

In [ ]:
# Select an analysis region 

# Mau Forest - Western Kenya
latitude = (0.0762, 0.21307)
longitude = (35.4034, 35.56681)

# Nairobi, Kenya
# latitude = (-1.4347, -1.151) 
# longitude = (36.6985, 36.9978) 

# Dar es Salaam, Tanzania
# latitude = (-7.0, -6.7)
# longitude = (39.1, 39.4)


**Visualize the selected area**

In [ ]:
display_map(longitude,latitude)

## <span id="define_extents">Define the Temporal Extents of the Analysis [&#9652;](#top)</span>  

<p style="color:red";><b>CHANGE INPUTS BELOW

In [ ]:
# Select the start and end periods for your analysis products
# The datetime function is (Year,Month,Day)
# These time windows will be used to make a mosaic, so typically pick a year length or more
# Be sure to evaluate the RGB mosaics to affirm they are not full of clouds

# Select the baseline time period (start and end)
baseline_time_period = ('2014-01-01', '2014-06-30')

# Select the analysis time period (start and end)
analysis_time_period = ('2017-01-01', '2017-06-30')

# Select the cloud-free mosaic type
# Options are: max_ndvi, min_ndvi, median, most_recent_pixel, geomedian
# If a geomedian is selected, it will take much longer to process
# It is most common to use the max_ndvi or median for these land change analyses
# HINT: Consider max_ndvi mosaics for NDVI analyses and median mosaics for EVI analyses 

baseline_mosaic_function = "median" 
analysis_mosaic_function = "median" 

## <span id="load_data">Load, Clean, and Mosaic Data from the Data Cube [&#9652;](#top)</span>

>#### Load Data (Baseline, Analysis) 

In [ ]:
#define common paramiters for all data loads: Location, Bands, Product, CRS and Resolution. Different loads will load different date ranges

common_load_params = dict(y = latitude, x = longitude,
                          measurements = ["red", "green", "blue", "nir", "swir1", "swir2", "pixel_qa"],
                          product = 'ls8_usgs_sr_scene',
                          output_crs='EPSG:6933',
                          resolution=(-30,30))

In [ ]:
#load data for baseline period
baseline_ds = dc.load(**common_load_params, time = baseline_time_period, dask_chunks = {'x':250, 'y':250})
print(baseline_ds)

In [ ]:
#load data for analis period
analysis_ds = dc.load(**common_load_params, time = analysis_time_period, dask_chunks = {'x':250, 'y':250})
print(analysis_ds)

> #### Clean Data
> Generating boolean masks that highlight valid pixels.
> Pixels must be cloud-free over land or water to be kept.

In [ ]:
unpack_functions = {"LANDSAT_8": ls8_unpack_qa}
unpack_function = unpack_functions['LANDSAT_8']

In [ ]:
def clean_mask(ds, unpacking_func, cover_types):
    masks = [unpacking_func(ds, cover_type) for cover_type in cover_types]
    return np.logical_or(*masks).values

In [ ]:
baseline_clean_mask = clean_mask(baseline_ds.pixel_qa, unpack_function, ["clear", "water"])
analysis_clean_mask = clean_mask(analysis_ds.pixel_qa, unpack_function, ["clear", "water"])

In [ ]:
baseline_ds = baseline_ds.where(baseline_clean_mask)
analysis_ds = analysis_ds.where(analysis_clean_mask)

### Compute dask arrays

In [ ]:
baseline_ds = baseline_ds.compute()

In [ ]:
analysis_ds = analysis_ds.compute()

>#### Mosaic
> Use clean masks in a time series composite

In [ ]:
mosaic_function = {"median": create_median_mosaic,
                   "max_ndvi": create_max_ndvi_mosaic,
                   "min_ndvi": create_min_ndvi_mosaic,
                   "geomedian": create_hdmedians_multiple_band_mosaic,
                   "most_recent_pixel": create_mosaic}

In [ ]:
baseline_compositor = mosaic_function[baseline_mosaic_function]
analysis_compositor = mosaic_function[analysis_mosaic_function]

In [ ]:
baseline_composite = baseline_compositor(baseline_ds, clean_mask = baseline_clean_mask)
analysis_composite = analysis_compositor(analysis_ds, clean_mask = analysis_clean_mask)

In [ ]:
rgb(baseline_composite)

In [ ]:
rgb(analysis_composite)


## <span id="calculate">Calculate Spectral Indices and Fractional Cover [&#9652;](#top)</span>  

In [ ]:
def NDVI(dataset):
    return (dataset.nir - dataset.red)/(dataset.nir + dataset.red)

In [ ]:
def EVI(ds, G=2.5, C1=6, C2=7.5, L=1, normalize=True):
    evi = G * (ds.nir - ds.red) / (ds.nir + C1 * ds.red - C2 * ds.blue + L)
    # Clamp values to the range [-1,2.5].
    evi.values[evi.values < -1] = -1
    evi.values[2.5 < evi.values] = 2.5
    if normalize:
        # Scale values in the  range [0,2.5] to the range [0,1].
        pos_vals_mask = 0 < evi.values
        evi.values[pos_vals_mask] = np.interp(evi.values[pos_vals_mask], (0, 2.5), (0,1))
    return evi

In [ ]:
ndvi = NDVI(baseline_composite) 
evi = EVI(baseline_composite)

In [ ]:
#change Coordinate names from x,y to Longitude, Latitude to be compatible with fractional cover module
baseline_composite_frac = baseline_composite.rename(name_dict={'x':'longitude','y':'latitude'})

frac_classes = frac_coverage_classify(baseline_composite_frac, clean_mask = np.ones(baseline_composite.pixel_qa.shape).astype(np.bool)) 

<p style="color:red";><b>CHANGE INPUTS BELOW

In [ ]:
# Select a spectral index to plot by removing one of the comment tags (#) below.
# Only 1 of the plotting statements should be uncommented.
# Adjust the scale of the output using vmin and vmax.

# (ndvi).plot(figsize=(10,10),cmap = "Greens", vmin=0.3, vmax=1.0)
# (evi).plot(figsize=(10,10),cmap = "Greens", vmin=0.3, vmax=1.0)
(frac_classes.pv).plot(figsize=(10,10),cmap = "Greens", vmin=30.0, vmax=100.0)

plt.show()

## <span id="threshold">Show Threshold Image [&#9652;](#top)</span>  

<p style="color:red";><b>CHANGE INPUTS BELOW

In [ ]:
# Select a baseline threshold range for a given parameter ... which is basically a "mask"
# The analysis will only consider pixels in this range for change detection
# No data or cloud pixels will also be masked in this process

# If you want to see all change, you should select the full range of values for each parameter
# NDVI full range = -1.0 to 1.0
# EVI full range = -1.0 to 1.0
# Fractional Cover full range = 0.0 to 100.0

# If you want to mask out everything but dense vegetation, typical of forested land, use the values below
# NDVI: 0.7 to 1.0
# EVI: 0.7 to 1.0
# Fractional Cover PV: 70.0 to 100.0

baseline_threshold_range = _min, _max = (70.0, 100.0) 

<p style="color:red";><b>CHANGE INPUTS BELOW

In [ ]:
# Select the parameter to be used for thresholding ... NDVI, EVI or FC
# Remove the comment statement to use the desired parameter

# baseline_filter_mask = np.logical_and(NDVI(baseline_composite) > _min, NDVI(baseline_composite) < _max)    
# baseline_filter_mask = np.logical_and(EVI(baseline_composite) > _min, EVI(baseline_composite) < _max)    
baseline_filter_mask = np.logical_and(frac_classes.pv > _min, frac_classes.pv < _max)    

In [ ]:
def aspect_ratio_helper(ds, fixed_width = 12):
        y,x = ds.values.shape
        width = fixed_width
        height = y * (fixed_width / x)
        return (width, height)

In [ ]:
RdYlGn.set_bad('black',1.)
Greens.set_bad('black',1.)

In [ ]:
# This is the baseline threshold plot that shows GREEN pixels in the threshold range
# Pixels with values outside the threshold range are white 

plt.figure(figsize = aspect_ratio_helper(baseline_filter_mask)) 
baseline_filter_mask.plot(cmap = "Greens")
plt.show()

**Filter the baseline composite with the threshold mask.**

In [ ]:
baseline_composite = baseline_composite.where(baseline_filter_mask)

In [ ]:
baseline_composite

## <span id="anomalies">Show Anomalies [&#9652;](#top)</span>  

### Show Spectral Anomaly Product

<p style="color:red";><b>CHANGE INPUTS BELOW

In [ ]:
# Select the parameter to be used for the analysis
# You will select two lines for each parameter (baseline and analysis)

# Select these two lines for NDVI
# parameter_baseline_composite = NDVI(baseline_composite)
# parameter_analysis_composite = NDVI(analysis_composite)

# Select these two lines for EVI
# parameter_baseline_composite = EVI(baseline_composite)
# parameter_analysis_composite = EVI(analysis_composite)

# Select these two lines for Fractional Cover (FC)
parameter_baseline_composite = frac_coverage_classify(baseline_composite, clean_mask = np.ones(baseline_composite.pixel_qa.shape).astype(np.bool)).pv
parameter_analysis_composite = frac_coverage_classify(analysis_composite, clean_mask = np.ones(analysis_composite.pixel_qa.shape).astype(np.bool)).pv 

In [ ]:
parameter_anomaly = parameter_analysis_composite - parameter_baseline_composite

In [ ]:
# Plot the parameter change from the baseline to the analysis period
# Significant loss in vegetation will be shown in RED
# Gains in vegetation will be shown in GREEN

plt.figure(figsize = aspect_ratio_helper(parameter_anomaly)) 
parameter_anomaly.plot(cmap = RdYlGn)
plt.show()

### Show Threshold Spectral Anomaly Product

<p style="color:red";><b>CHANGE INPUTS BELOW

In [ ]:
# Select an Anomaly Threshold Range to filter the results from the previous image
# Be sure to put the smallest value in the "minimum_change" location (be careful of negative values)

# Losses are typically in these ranges for deforestation
# NDVI: -0.2 to -0.7
# EVI: -0.5 to -1.75
# Fractional Cover PV: -20 to -70

minimum_change = -70
maximum_change = -20


In [ ]:
# Create a threshold plot using the MIN and MAX range defined above

def threshold_plot(da, min_threshold, max_threshold, mask = None, width = 10, *args, **kwargs): 
    color_in    = np.array([255,0,0])
    color_out   = np.array([0,0,0])
    color_cloud = np.array([255,255,255])
    
    array = np.zeros((*da.values.shape, 3)).astype(np.int16)
    
    inside  = np.logical_and(da.values > min_threshold, da.values < max_threshold)
    outside = np.invert(inside)
    masked  = np.zeros(da.values.shape).astype(bool) if mask is None else mask
    
    array[inside] =  color_in
    array[outside] = color_out
    array[masked] =  color_cloud

    def figure_ratio(ds, fixed_width = 10):
        width = fixed_width
        height = len(ds.latitude) * (fixed_width / len(ds.longitude))
        return (width, height)


    fig, ax = plt.subplots(figsize = figure_ratio(da,fixed_width = width))
    
    lat_formatter = FuncFormatter(lambda y_val, tick_pos: "{0:.3f}".format(da.latitude.values[tick_pos] ))
    lon_formatter = FuncFormatter(lambda x_val, tick_pos: "{0:.3f}".format(da.longitude.values[tick_pos]))

    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    
    plt.title("Threshold: {} < x < {}".format(min_threshold, max_threshold))
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    
    plt.imshow(array, *args, **kwargs)
    plt.show()

In [ ]:
no_data_mask = np.logical_or(np.isnan(baseline_composite.red.values), np.isnan(analysis_composite.red.values)) 

In [ ]:
# Plot the parameter change using thresholds and 3 colors 
# RED = data in the threshold range (significant land change)
# BLACK = data outside the threshold range (no significant change)
# WHITE = data outside the baseline mask (not dense vegetation) or clouds (no data)

threshold_plot(parameter_anomaly, minimum_change, maximum_change, mask = no_data_mask, width  = 10)

In [ ]:
def threshold_count(da, min_threshold, max_threshold, mask = None):
    def count_not_nans(arr):
        return np.count_nonzero(~np.isnan(arr))
    
    in_threshold = np.logical_and( da.values > min_threshold, da.values < max_threshold)
    
    total_non_cloudy = count_not_nans(da.values) if mask is None else np.sum(mask) 
    
    return dict(total = np.size(da.values),
                total_non_cloudy = total_non_cloudy,
                inside = np.nansum(in_threshold),
                outside = total_non_cloudy - np.nansum(in_threshold)
               )    
    
def threshold_percentage(da, min_threshold, max_threshold, mask = None):
    counts = threshold_count(da, min_threshold, max_threshold, mask = mask)
    return dict(percent_inside_threshold = (counts["inside"]   / counts["total"]) * 100.0,
                percent_outside_threshold = (counts["outside"] / counts["total"]) * 100.0,
                percent_clouds = ( 100.0-counts["total_non_cloudy"] / counts["total"] * 100.0))


In [ ]:
# This output is a count of the pixels that fall within each threshold range

threshold_count(parameter_anomaly,minimum_change,maximum_change)

In [ ]:
# This output is a percentage of the pixels that fall within each threshold range

threshold_percentage(parameter_anomaly,minimum_change,maximum_change)

## <span id="export">Create GeoTIFF Output Products [&#9652;](#top)</span>  

<p style="color:red";><b>CHANGE INPUTS BELOW

In [ ]:
from utils.data_cube_utilities.dc_utilities import write_geotiff_from_xr

# Remove the comment tag # to create a GeoTIFF output product
# Change the name of the output file, or it will be overwritten for each run 
# There are 3 possible outputs ... baseline mosaic, analysis mosaic, or anomaly product

# write_geotiff_from_xr("geotiffs/sample_baseline_01.tif", bands = ['red','green','blue','nir','swir1','swir2'], baseline_composite)
# write_geotiff_from_xr("geotiffs/sample_analysis_01.tif", bands = ['red','green','blue','nir','swir1','swir2'], analysis_composite)
write_geotiff_from_xr("../output/geotiffs/sample_anomaly_01.tif", parameter_anomaly.to_dataset(name='pv'), bands=['pv'])

In [ ]:
!ls -lah ../output/geotiffs/*.tif

## Notes on modifications made moving from ARDC to DE Africa

Started modifying this notebook for DE Africa sandbox. However the process **Filter the baseline composite with the threshold mask.** uses so much memory that the notebook overloads the Kernel, so this code would only be able to run over a very very small area, unless it was modified to use Dask. 

Changes made

- Replace dc.load fuction with DE Africa load
- Move all module imports to start of notebook
- Removed code to view extent of datacube replaced with mapviewer that can be used to visualise spatial extent of data
- Changed rgb plot fuction to DE Africa rgb plot fuction

Changes not made:

- replaced save to geotiff fuction with DE Africa fuction